<a href="https://colab.research.google.com/github/jessicaMorale/RI-busquedaB-MoralesJ/blob/main/Examen02/Examen02_Morales_Jessica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Examen Segundo Bimestre**
## Recuperación de la infromación


*   Morales Jessica
*   GR1CC



In [35]:
# Actualizar pip
!pip install --upgrade pip

!pip install pymupdf faiss-cpu sentence-transformers nltk requests python-dotenv


In [36]:
from google.colab import files

# Llamar a este comando te permitirá seleccionar archivos desde tu dispositivo
uploaded = files.upload()

# Verificar los archivos subidos
for file_name in uploaded.keys():
    print(f"Archivo {file_name} subido exitosamente!")


Saving .env to .env (2)
Archivo .env (2) subido exitosamente!


In [37]:
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la API Key de Gemini desde el archivo .env
api_key = os.getenv("GEMINI_API_KEY")

# Verificar si la clave API fue cargada correctamente
if api_key:
    print("API Key cargada exitosamente.")
else:
    print("No se pudo cargar la API Key. Asegúrate de tener el archivo .env con la clave correcta.")


API Key cargada exitosamente.


In [38]:
import requests

# Función para realizar la consulta a la Gemini API
def query_gemini_api(query_text):
    url = "https://api.gemini.com/generate"  # URL base para la API de Gemini
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "query": query_text
    }

    # Realizamos la solicitud a Gemini API
    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        return response.json()  # Retorna la respuesta como JSON
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


In [39]:
from google.colab import files

# Llamar a este comando te permitirá seleccionar archivos desde tu dispositivo
uploaded = files.upload()

# Verificar los archivos subidos
for file_name in uploaded.keys():
    print(f"Archivo {file_name} subido exitosamente!")

Saving arxiv_examen2.json to arxiv_examen2 (1).json
Archivo arxiv_examen2 (1).json subido exitosamente!


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [40]:
import pandas as pd

# Cargar el dataset JSON
dataset_path = "arxiv_examen2.json"
df = pd.read_json(dataset_path, lines=True)

# Mostrar las primeras filas del dataset
print(df.head())


         id                                              title  \
0  704.0001  Calculation of prompt diphoton production cros...   
1  704.0002           Sparsity-certifying Graph Decompositions   
2  704.0003  The evolution of the Earth-Moon system based o...   
3  704.0004  A determinant of Stirling cycle numbers counts...   
4  704.0005  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                            abstract  
0    A fully differential calculation in perturba...  
1    We describe a new algorithm, the $(k,\ell)$-...  
2    The evolution of Earth-Moon system is descri...  
3    We show that a determinant of Stirling cycle...  
4    In this paper we show how to compute the $\L...  


In [41]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Inicializar el stemmer y las stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Función de preprocesamiento
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizar el texto y convertirlo a minúsculas
    return [stemmer.stem(t) for t in tokens if t.isalpha() and t not in stop_words]

# Aplicar el preprocesamiento a los abstracts
df['processed_text'] = df['abstract'].apply(preprocess)

# Verificar los datos procesados
print(df[['title', 'processed_text']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                               title  \
0  Calculation of prompt diphoton production cros...   
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                      processed_text  
0  [fulli, differenti, calcul, perturb, quantum, ...  
1  [describ, new, algorithm, k, game, color, use,...  
2  [evolut, system, describ, dark, matter, field,...  
3  [show, determin, stirl, cycl, number, count, u...  
4  [paper, show, comput, norm, use, dyadic, grid,...  


In [42]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo para generar embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para el texto procesado
embeddings = model.encode(df['processed_text'].tolist(), convert_to_numpy=True)

# Crear el índice FAISS para realizar la búsqueda de documentos relevantes
import faiss

index = faiss.IndexFlatL2(embeddings.shape[1])  # Distancia L2
index.add(embeddings)

# Verificar que el índice está entrenado
print(index.is_trained)


True


In [43]:
def search(query, top_k=5):
    # Procesar la consulta
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Buscar en el índice FAISS
    distances, indices = index.search(query_embedding, top_k)

    # Extraer los documentos más relevantes
    relevant_docs = df.iloc[indices[0]]

    # Mostrar los documentos relevantes
    return relevant_docs[['title', 'abstract']]

# Ejemplo de consulta
query = "machine learning algorithms"
results = search(query)
print(results)


                                                   title  \
15008            Computing the complete CS decomposition   
5347   Degree Optimization and Stability Condition fo...   
7229   Computability of simple games: A characterizat...   
10626  Efficient method for detection of periodic orb...   
9737   Improved Neural Modeling of Real-World Systems...   

                                                abstract  
15008    An algorithm is developed to compute the com...  
5347     The min-sum (MS) algorithm is arguably the s...  
7229     The class of algorithmically computable simp...  
10626    An algorithm for detecting unstable periodic...  
9737     Neural network models of real-world systems,...  


In [44]:
!pip install google-genai


In [45]:
from google import genai
import os

# Inicializamos el cliente de Gemini utilizando la API Key
api_key = os.getenv("GEMINI_API_KEY")  # Se asume que la clave está en el archivo .env

client = genai.Client(api_key=api_key)

# Función para realizar una consulta y generar contenido con Gemini API
def generate_response_with_gemini(prompt, model="gemini-2.5-flash"):
    try:
        # Realizar la consulta usando el cliente de Gemini
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )

        # Verificar si la respuesta fue exitosa y devolver el texto generado
        if response:
            return response.text  # Devuelve el texto generado por Gemini
        else:
            return "No se pudo generar la respuesta."

    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"

# Ejemplo de uso
query = "What is machine learning?"
# Crear un prompt ficticio para la consulta (ajusta este contexto según tu aplicación)
prompt = f"Provide a short definition and example of {query}."

# Generar la respuesta con Gemini
response_text = generate_response_with_gemini(prompt)
print("Respuesta generada por Gemini:", response_text)


Respuesta generada por Gemini: **Definition:**
Machine learning (ML) is a branch of artificial intelligence where computer systems **learn from data** to identify patterns, make predictions, or improve performance on a task **without being explicitly programmed** for every specific rule. Instead of following fixed instructions, they build models based on observed data.

**Example:**
**Email Spam Filter:**
Instead of a programmer writing countless "if-then" rules (e.g., "if 'lottery' and 'win', then spam"), an ML system is **fed thousands of emails labeled as spam or not spam**. It then **learns patterns** from this data (e.g., common words, sender details, email structure) to **predict** whether a new, unseen email is spam. It gets better at this prediction over time as it processes more emails and learns from your feedback (marking something as "not spam").


In [46]:
import pandas as pd

# Función para realizar la búsqueda con BM25
def search_bm25(query, top_k=5):
    query_tokens = query.lower().split()  # Tokenizar la consulta
    bm25_results = df[df['processed_text'].apply(lambda x: any(token in x for token in query_tokens))]  # Búsqueda simple
    return bm25_results.head(top_k)

# Función para realizar la búsqueda con TF-IDF
def search_tfidf(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)  # Codificar la consulta
    distances, indices = index.search(query_embedding, top_k)  # Buscar en el índice FAISS
    tfidf_results = df.iloc[indices[0]]  # Obtener los documentos relevantes
    return tfidf_results[['title', 'abstract']]  # Devolver los títulos y abstracts

# Función para realizar la búsqueda con Embeddings
def search_embeddings(query, top_k=5):
    # Codificar la consulta usando el modelo de embeddings
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Buscar en el índice FAISS de embeddings
    distances, indices = index.search(query_embedding, top_k)

    # Extraer los documentos más relevantes
    embedding_results = df.iloc[indices[0]]

    return embedding_results[['title', 'abstract']]  # Devolver los títulos y abstracts

# Función para generar la tabla comparativa entre BM25, TF-IDF, y Embedding
def compare_responses(query, top_k=5):
    # Obtener resultados de BM25
    bm25_results = search_bm25(query, top_k)

    # Obtener resultados de TF-IDF
    tfidf_results = search_tfidf(query, top_k)

    # Obtener resultados de Embeddings
    embedding_results = search_embeddings(query, top_k)

    # Asegurarse de que todos los resultados tengan la misma longitud
    max_len = max(len(bm25_results), len(tfidf_results), len(embedding_results), top_k)

    # Rellenar con valores vacíos en caso de que haya menos resultados de BM25, TF-IDF o Embedding
    if len(bm25_results) < max_len:
        bm25_results = pd.concat([bm25_results, pd.DataFrame([{"title": "", "abstract": ""}] * (max_len - len(bm25_results)))]).reset_index(drop=True)

    if len(tfidf_results) < max_len:
        tfidf_results = pd.concat([tfidf_results, pd.DataFrame([{"title": "", "abstract": ""}] * (max_len - len(tfidf_results)))]).reset_index(drop=True)

    if len(embedding_results) < max_len:
        embedding_results = pd.concat([embedding_results, pd.DataFrame([{"title": "", "abstract": ""}] * (max_len - len(embedding_results)))]).reset_index(drop=True)

    # Crear el DataFrame comparativo
    comparison_df = pd.DataFrame({
        'Query': [query] * max_len,
        'BM25 Titles': bm25_results['title'].tolist(),
        'BM25 Abstracts': bm25_results['abstract'].tolist(),
        'TF-IDF Titles': tfidf_results['title'].tolist(),
        'TF-IDF Abstracts': tfidf_results['abstract'].tolist(),
        'Embedding Titles': embedding_results['title'].tolist(),
        'Embedding Abstracts': embedding_results['abstract'].tolist()
    })

    return comparison_df

# Realizar la consulta y obtener la tabla comparativa
query = "machine learning algorithms"
result_df = compare_responses(query)

# Visualizar la tabla comparativa
import IPython.display as display
display.display(result_df)

# Explicar los resultados de la evaluación
def explain_results(df):
    for index, row in df.iterrows():
        print(f"\nQuery: {row['Query']}")
        print(f"BM25 Response:\nTitle: {row['BM25 Titles']}\nAbstract: {row['BM25 Abstracts']}")
        print(f"TF-IDF Response:\nTitle: {row['TF-IDF Titles']}\nAbstract: {row['TF-IDF Abstracts']}")
        print(f"Embedding Response:\nTitle: {row['Embedding Titles']}\nAbstract: {row['Embedding Abstracts']}")
        print("\n--- Evaluation ---")

        # Evaluación simple de la respuesta más relevante (puedes agregar una lógica más avanzada según el caso)
        if row['BM25 Abstracts'] == row['Embedding Abstracts'] and row['Embedding Abstracts'] == row['TF-IDF Abstracts']:
            print("All models are giving similar responses.")
        elif row['BM25 Abstracts'] == row['Embedding Abstracts']:
            print("BM25 and Embedding are providing similar information.")
        elif row['Embedding Abstracts'] == row['TF-IDF Abstracts']:
            print("Embedding and TF-IDF are providing similar information.")
        else:
            print("The responses from all models are different, indicating the models' varied effectiveness.")

        print("\n=====================================")

# Llamar a la función para explicar los resultados
explain_results(result_df)


,Query,BM25 Titles,BM25 Abstracts,TF-IDF Titles,TF-IDF Abstracts,Embedding Titles,Embedding Abstracts
0,machine learning algorithms,,,Computing the complete CS decomposition,An algorithm is developed to compute the com...,Computing the complete CS decomposition,An algorithm is developed to compute the com...
1,machine learning algorithms,,,Degree Optimization and Stability Condition fo...,The min-sum (MS) algorithm is arguably the s...,Degree Optimization and Stability Condition fo...,The min-sum (MS) algorithm is arguably the s...
2,machine learning algorithms,,,Computability of simple games: A characterizat...,The class of algorithmically computable simp...,Computability of simple games: A characterizat...,The class of algorithmically computable simp...
3,machine learning algorithms,,,Efficient method for detection of periodic orb...,An algorithm for detecting unstable periodic...,Efficient method for detection of periodic orb...,An algorithm for detecting unstable periodic...
4,machine learning algorithms,,,Improved Neural Modeling of Real-World Systems...,"Neural network models of real-world systems,...",Improved Neural Modeling of Real-World Systems...,"Neural network models of real-world systems,..."



Query: machine learning algorithms
BM25 Response:
Title: 
Abstract: 
TF-IDF Response:
Title: Computing the complete CS decomposition
Abstract:   An algorithm is developed to compute the complete CS decomposition (CSD) of a
partitioned unitary matrix. Although the existence of the CSD has been
recognized since 1977, prior algorithms compute only a reduced version (the
2-by-1 CSD) that is equivalent to two simultaneous singular value
decompositions. The algorithm presented here computes the complete 2-by-2 CSD,
which requires the simultaneous diagonalization of all four blocks of a unitary
matrix partitioned into a 2-by-2 block structure. The algorithm appears to be
the only fully specified algorithm available. The computation occurs in two
phases. In the first phase, the unitary matrix is reduced to bidiagonal block
form, as described by Sutton and Edelman. In the second phase, the blocks are
simultaneously diagonalized using techniques from bidiagonal SVD algorithms of
Golub, Kahan, a

In [47]:
while True:
    query = input("\nIngresa tu pregunta (o escribe 'salir' para terminar): ")
    if query.lower() in ['salir', 'exit', 'quit']:
        print("Sesión finalizada.")
        break

    # Generar respuesta con Gemini
    response = generate_response_with_gemini(query)
    print("\nRespuesta del modelo:")
    print(response)



Ingresa tu pregunta (o escribe 'salir' para terminar): system

Respuesta del modelo:
The word "system" can refer to many different things! To help me understand what you're looking for, could you please clarify?

For example, are you asking about:

*   **Me** (as an AI language model system)?
*   The general **concept of a system** (e.g., a computer system, an ecological system, a social system)?
*   A **specific system** you have in mind?
*   Or something else entirely?

Ingresa tu pregunta (o escribe 'salir' para terminar): computing

Respuesta del modelo:
"Computing" is a wonderfully broad and fundamental term! Given just the word, I'll provide a general overview of what it encompasses.

At its core, **computing** refers to:

1.  **The act of calculation or processing information:** This is the most literal sense, stemming from its origins in performing arithmetic.
2.  **The study of algorithms and information processes:** This is the academic field of **Computer Science**, which e